# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [2]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL_GPT = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [4]:
# here is the question; type over this to ask something new
example_question = """
Please explain what this code does and why:
"""

example_snippet = """
yield from {book.get("author") for book in books if book.get("author")}
"""

# tutor system prompt
tutor_system_prompt = f"""
You are a an enterprise architect at a MAG7 company that explains code.
You are given a question and a code snippet.
You need to explain the code snippet to senior software engineers and technical leads/staff engineers. Make sure to provide concise and clear explanations. You may also be asked to refactor or optimize code that is not well written. If so, you should do so using the latest industry best practices. You should avoid taking shortcuts and instead focus on writing correct, robust, and efficient code that is easy to maintain and scale. You should respond in markdown format.
Here is an example question and snippet:

{example_question}
{example_snippet}
"""

In [5]:
#user prompt
question = """
what does this code do and why? Can you optimize it?
"""

snippet = """
response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]
"""

tutor_user_prompt = f"""
{question}
{snippet}
"""

In [6]:
# Get gpt-4o-mini to answer, with streaming
messages = [
    {"role": "system", "content": tutor_system_prompt},
    {"role": "user", "content": tutor_user_prompt}
]

def stream_response():
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_response()